In [127]:
import pandas as pd
import numpy as np
import numpy.typing as npt
import math

In [128]:
def entropy(value_count: npt.ArrayLike) -> float:
    if 0 in value_count:
        return 0
    e = 0.0
    total = sum(value_count)
    for vc in value_count:
        e -= vc / total * math.log2(vc / total)
    return e

In [129]:
def gain(df: pd.DataFrame, attr: str, target: str) -> float:
    s = df[target].value_counts()
    g = entropy(s.array)
    sv = df[attr].value_counts()
    for at in sv.index:
        g -= sv[at] / len(df) * entropy(df[df[attr] == at][target].value_counts().array) 
    return g

In [130]:
df = pd.read_csv("../data/playtennis.csv")

df

,Day,Outlook,Temperature,Humidity,Wind,Play Tennis
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes
5,6,Rain,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rain,Mild,Normal,Weak,Yes


In [131]:
gain(df, "Wind", "Play Tennis")


0.048127030408269544